In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from scipy import sparse
from tqdm import tqdm
import matplotlib.pyplot as plt
import gc
import printTime as pt

In [2]:
pt.printTime()

2019-08-21 07:44:08


# 计算usage中train和test共同拥有的app

In [ ]:
usage = pd.read_csv(r'../data/user_app_usage.csv',header=None,usecols=[0,1])
pt.printTime()

age_train = pd.read_csv(r'../data/age_train.csv',header=None)
age_test =pd.read_csv(r'../data/age_test.csv',header=None)

usage_train = pd.merge(age_train,usage,how='inner',left_on=0,right_on=0)
usage_test = pd.merge(age_test,usage,how='inner',left_on=0,right_on=0)
pt.printTime()

app_train = usage_train.drop_duplicates(subset=['1_y'])['1_y']
app_test = usage_test.drop_duplicates(subset=[1])[1]
pt.printTime()

del usage_train,usage_test
gc.collect()

commonAPP = list(set(app_train.values).intersection(set(app_test.values)))

del app_train,app_test
gc.collect()

common_app = pd.DataFrame(commonAPP)

common_app.to_csv(r'../processed/train_test_commonAPP.csv',header=None,index=False)

del common_app
gc.collect()

bool_index = usage[1].isin(commonAPP)
usage = pd.DataFrame(usage[bool_index])

# usage.to_csv(r'../processed/commonAPP_usage.csv',header=None,index=False)
pt.printTime()

# usage词频——数据准备

In [ ]:
# usage = pd.read_csv(r'../processed/commonAPP_usage.csv',header=None)
print(usage.shape)
print(usage.head())

In [ ]:
user_num = usage[0].value_counts()

In [ ]:
fig=plt.figure(figsize=(20,4))
plt.plot(range(len(user_num)),user_num)
plt.show()

In [ ]:
gc.collect()

In [ ]:
# usage = usage.values
first_flag = True
user = [] #[uId,app_string]
usage_str = []
app_str = ''
usage = usage.values
for i in tqdm(range(usage.shape[0])):
    if first_flag == True:
        user.append(usage[i][0])
        app_str = usage[i][1]
        first_flag = False
    else:
        if usage[i][0] == user[0]:
            app_str = app_str + ' ' + usage[i][1]
        else:
            usage_str.append([user[0],app_str])
            user = []
            user.append(usage[i][0])
            app_str = usage[i][1]
    if i == usage.shape[0]-1:
        usage_str.append([user[0],app_str])

In [ ]:
pt.printTime()
usage_str = pd.DataFrame(usage_str)
print(usage_str.shape)
print(usage_str.head())
pt.printTime()

In [ ]:
pt.printTime()
usage_str.to_csv(r'../processed/usage_str.csv',header=None,index=False)
pt.printTime()

In [ ]:
del usage
gc.collect()

# 特征生成

In [12]:
usage_str = pd.read_csv(r'../processed/usage_str.csv',header=None)
print(usage_str.shape)
print(usage_str.head())

(1992583, 2)
         0                                                  1
0  1000364  a00289850 a00287085 a00289826 a00287085 a00278...
1  1000387  a00187480 a00289791 a00234796 a00187480 a00442...
2  1000872  a00137119 a00310778 a00310778 a00289791 a00289...
3  1000951  a0048332 a00289826 a00348591 a00248909 a002489...
4  1001091  a00292472 a00348507 a00443258 a00289826 a00277...


In [13]:
important_app = np.loadtxt('../processed/important_app_usage.txt', dtype=np.str, delimiter=',')
important_app.shape

(488124,)

In [14]:
commonAPP = pd.read_csv(r'../processed/train_test_commonAPP.csv',header=None)
commonAPP = set(commonAPP[0].values)

In [15]:
important_app_list = []
for i in tqdm(range(len(important_app))):
    if important_app[i] in commonAPP:
        important_app_list.append(important_app[i])

100%|██████████| 488124/488124 [00:00<00:00, 971569.33it/s]


In [16]:
voca = {}
num = 10000
for i in tqdm(range(num)):
    voca[important_app_list[i]] = i

100%|██████████| 10000/10000 [00:00<00:00, 1164987.36it/s]


In [17]:
age_train = pd.read_csv(r'../data/age_train.csv',header=None,usecols=[0])
age_test = pd.read_csv(r'../data/age_test.csv',header=None)

In [18]:
test = pd.merge(age_test,usage_str,how='inner',on=0)
test = pd.merge(age_test,test,how='outer',on=0)
print(test.shape)
print(test.head(10))

(502500, 2)
         0                                                  1
0  1000002  a00287085 a00170443 a00278858 a00287085 a00928...
1  1000003  a00289826 a00289791 a00263057 a00178623 a00278...
2  1000004  a00333490 a00336224 a00287085 a00272228 a00281...
3  1000005                                                NaN
4  1000006                                                NaN
5  1000007  a00287085 a00180973 a0092898 a00263531 a004856...
6  1000008  a0021880 a00263057 a0017615 a009932 a00141414 ...
7  1000009  a0036392 a0071866 a0027551 a00289791 a00170432...
8  1000010                                                NaN
9  1000012  a00289826 a0048562 a00289826 a00157220 a002898...


In [19]:
train = pd.merge(age_train,usage_str,how='inner',on=0)
train = pd.merge(age_train,train,how='outer',on=0)
print(train.shape)
print(train.head(10))

(2010000, 2)
         0                                                  1
0  1000001  a00867 a00289791 a00289826 a00289791 a0011560 ...
1  1000011                                                NaN
2  1000015  a0032847 a00170432 a00170432 a00247519 a001704...
3  1000019                                                NaN
4  1000023  a00278905 a00289791 a00290038 a00289791 a00289...
5  1000025  a00290027 a00102782 a00445313 a00289791 a00144...
6  1000029  a0022093 a00289659 a00290037 a00287085 a002897...
7  1000035  a00289850 a00157201 a00289791 a00157201 a00287...
8  1000037  a00289866 a00289791 a00290037 a00289600 a00218...
9  1000038  a00103666 a00365869 a00336224 a00336239 a00287...


In [20]:
from sklearn.impute import SimpleImputer

In [21]:
imp = SimpleImputer(strategy='constant',fill_value=' ')
train = imp.fit_transform(train.values)
test = imp.fit_transform(test.values)

In [22]:
data = np.vstack((train,test))

In [23]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [24]:
tfidf_vec = TfidfVectorizer(vocabulary=voca) 
tfidf_matrix = tfidf_vec.fit_transform(data[:,1])

In [25]:
tfidf_matrix.shape

(2512500, 10000)

In [26]:
train = tfidf_matrix[:train.shape[0],:]
test = tfidf_matrix[train.shape[0]:,:]

In [27]:
sparse.save_npz(r'../trainTestData/trainData10000.npz',train)
sparse.save_npz(r'../trainTestData/testData10000.npz',test)

In [28]:
data = pd.DataFrame(train[:1000,:].toarray())

In [29]:
print(data.head())
pt.printTime()

       0     1     2         3     4     5         6     7     8     9     \
0  0.179791   0.0   0.0  0.000000   0.0   0.0  0.000000   0.0   0.0   0.0   
1  0.000000   0.0   0.0  0.000000   0.0   0.0  0.000000   0.0   0.0   0.0   
2  0.090465   0.0   0.0  0.560534   0.0   0.0  0.000000   0.0   0.0   0.0   
3  0.000000   0.0   0.0  0.000000   0.0   0.0  0.000000   0.0   0.0   0.0   
4  0.023668   0.0   0.0  0.439950   0.0   0.0  0.303903   0.0   0.0   0.0   

   ...  9990  9991  9992  9993  9994  9995  9996  9997  9998  9999  
0  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
1  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
3  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
4  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[5 rows x 10000 columns]
2019-08-21 01:08:35
